<a href="https://colab.research.google.com/github/vngeno/E-COMMERCE-RECOMMENDATION-SYSTEM-/blob/Content-Based-Filtering/Content_Recommendation_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np

# For Text
import re
from sklearn.feature_extraction.text import TfidfVectorizer

# Capture similarity 
from sklearn.metrics.pairwise import linear_kernel

In [3]:
df = pd.read_csv("/content/drive/Shareddrives/Team Stars/Data/df_clean.csv",index_col=0)
df.head(2)

,id,name,brand,primarycategories,manufacturernumber,reviews_dorecommend,reviews_numhelpful,reviews_rating,reviews_text,reviews_title,reviews_username
0,AVpgNzjwLJeJML43Kpxn,AmazonBasics AAA Performance Alkaline Batterie...,Amazonbasics,Health & Beauty,HL-002619,True,1.0,3,I order 3 of them and one of the item is bad q...,... 3 of them and one of the item is bad quali...,Byger yang
1,AVpgNzjwLJeJML43Kpxn,AmazonBasics AAA Performance Alkaline Batterie...,Amazonbasics,Health & Beauty,HL-002619,True,1.0,4,Bulk is always the less expensive way to go fo...,... always the less expensive way to go for pr...,ByMG


In [4]:
data=df[['brand','name','primarycategories','reviews_text']]
data=data.rename(columns={'brand':'type','name':'title','primarycategories':'listed_in','reviews_text':'description'})
data.head()

,type,title,listed_in,description
0,Amazonbasics,AmazonBasics AAA Performance Alkaline Batterie...,Health & Beauty,I order 3 of them and one of the item is bad q...
1,Amazonbasics,AmazonBasics AAA Performance Alkaline Batterie...,Health & Beauty,Bulk is always the less expensive way to go fo...
2,Amazonbasics,AmazonBasics AAA Performance Alkaline Batterie...,Health & Beauty,Well they are not Duracell but for the price i...
3,Amazonbasics,AmazonBasics AAA Performance Alkaline Batterie...,Health & Beauty,Seem to work as well as name brand batteries a...
4,Amazonbasics,AmazonBasics AAA Performance Alkaline Batterie...,Health & Beauty,These batteries are very long lasting the pric...


In [5]:
data['listed_in'] = [re.sub(r'[^\w\s]', '', t) for t in data['listed_in']]
data['description'] = [re.sub(r'[^\w\s]', '', t) for t in data['description']]

data['listed_in'] = [t.lower() for t in data['listed_in']]
data['description'] = [t.lower() for t in data['description']]
data.head()

,type,title,listed_in,description
0,Amazonbasics,AmazonBasics AAA Performance Alkaline Batterie...,health beauty,i order 3 of them and one of the item is bad q...
1,Amazonbasics,AmazonBasics AAA Performance Alkaline Batterie...,health beauty,bulk is always the less expensive way to go fo...
2,Amazonbasics,AmazonBasics AAA Performance Alkaline Batterie...,health beauty,well they are not duracell but for the price i...
3,Amazonbasics,AmazonBasics AAA Performance Alkaline Batterie...,health beauty,seem to work as well as name brand batteries a...
4,Amazonbasics,AmazonBasics AAA Performance Alkaline Batterie...,health beauty,these batteries are very long lasting the pric...


In [6]:
#Fit into the title
#Lets vectorize all these reviews
from sklearn.feature_extraction.text import CountVectorizer
#initialize vectorizer
vect = CountVectorizer(analyzer = 'word', ngram_range = (1,2), stop_words = 'english', min_df = 0.002) 
#min_df = rare words, max_df = most used words
#ngram_range = (1,2) - if used more than  1(value), lots of features or noise
vect.fit(data['description'])
title_matrix = vect.transform(data['description'])
title_matrix.shape

(28332, 1095)

In [7]:
#Lets find features
features = vect.get_feature_names()
features

['10',
 '10 year',
 '100',
 '12',
 '15',
 '16',
 '16gb',
 '20',
 '2nd',
 '30',
 '3rd',
 '48',
 '50',
 'aa',
 'aa batteries',
 'aaa',
 'aaa batteries',
 'aas',
 'ability',
 'able',
 'absolutely',
 'absolutely love',
 'absolutely loves',
 'access',
 'account',
 'actually',
 'add',
 'added',
 'addition',
 'additional',
 'ads',
 'adult',
 'adults',
 'advertised',
 'affordable',
 'age',
 'age appropriate',
 'ages',
 'ago',
 'alexa',
 'alkaline',
 'alkaline batteries',
 'allow',
 'allows',
 'alot',
 'alternative',
 'amazing',
 'amazon',
 'amazon account',
 'amazon app',
 'amazon basic',
 'amazon basics',
 'amazon batteries',
 'amazon brand',
 'amazon hd',
 'amazon kids',
 'amazon prime',
 'amazon tablet',
 'amazonbasics',
 'amazonbasics batteries',
 'amazons',
 'android',
 'annoying',
 'app',
 'app store',
 'appear',
 'apple',
 'applications',
 'appropriate',
 'apps',
 'apps available',
 'arent',
 'arrived',
 'ask',
 'asked',
 'audio',
 'available',
 'average',
 'avid',
 'away',
 'awesome',


In [8]:
#performing cosine similarity
from sklearn.metrics.pairwise import cosine_similarity
cosine_sim_titles = cosine_similarity(title_matrix, title_matrix)
cosine_sim_titles.shape

(28332, 28332)

In [9]:
#Get books which are similar to a given product
title_id = 20000
data['title'].iloc[title_id]

'All-New Fire HD 8 Kids Edition Tablet, 8 HD Display, 32 GB, Pink Kid-Proof Case'

In [10]:
#Find out what features have been considered  by the vectorizer for a given product ?
feature_array = np.squeeze(title_matrix[title_id].toarray()) #squeeze activity matrix into array
idx = np.where(feature_array > 0)
idx[0]
[features[x] for x in idx[0]]

['apps',
 'bought',
 'bought gift',
 'enjoy',
 'gift',
 'help',
 'learning',
 'old',
 'old son',
 'plenty',
 'price',
 'son',
 'year',
 'year old']

In [11]:
# Find index of feature
idx[0]

array([  69,  131,  134,  317,  391,  460,  551,  674,  680,  742,  760,
        895, 1083, 1084])

In [12]:
#Cosine similarity with other similar products
n = 15 #products to be recommended
top_n_idx = np.flip(np.argsort(cosine_sim_titles[title_id,]), axis = 0)[0:n]
top_n_sim_values = cosine_sim_titles[title_id, top_n_idx]
top_n_sim_values

array([1.        , 1.        , 0.60024505, 0.60024505, 0.56591646,
       0.56591646, 0.56591646, 0.56591646, 0.56591646, 0.56011203,
       0.56011203, 0.51449576, 0.50572174, 0.50572174, 0.5007734 ])

In [13]:
#find top n with values > 0
top_n_idx = top_n_idx[top_n_sim_values > 0]
#Matching products
tit=data['title'].iloc[top_n_idx]
tit

19999    All-New Fire HD 8 Kids Edition Tablet, 8 HD Di...
20000    All-New Fire HD 8 Kids Edition Tablet, 8 HD Di...
24464    Fire Kids Edition Tablet, 7 Display, Wi-Fi, 16...
13705    Fire Kids Edition Tablet, 7 Display, Wi-Fi, 16...
22351    Fire Tablet with Alexa, 7 Display, 16 GB, Blue...
15321    All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...
18654    Fire Tablet, 7 Display, Wi-Fi, 16 GB - Include...
26927    Fire HD 8 Tablet with Alexa, 8 HD Display, 16 ...
21444    Fire Tablet with Alexa, 7 Display, 16 GB, Mage...
17200    All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...
28043    Fire HD 8 Tablet with Alexa, 8 HD Display, 16 ...
20677    All-New Fire HD 8 Tablet with Alexa, 8 HD Disp...
17615    All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...
28289    Fire HD 8 Tablet with Alexa, 8 HD Display, 16 ...
15069    Fire Kids Edition Tablet, 7 Display, Wi-Fi, 16...
Name: title, dtype: object

In [14]:
tit.unique()

array(['All-New Fire HD 8 Kids Edition Tablet, 8 HD Display, 32 GB, Blue Kid-Proof Case',
       'All-New Fire HD 8 Kids Edition Tablet, 8 HD Display, 32 GB, Pink Kid-Proof Case',
       'Fire Kids Edition Tablet, 7 Display, Wi-Fi, 16 GB, Green Kid-Proof Case',
       'Fire Kids Edition Tablet, 7 Display, Wi-Fi, 16 GB, Pink Kid-Proof Case',
       'Fire Tablet with Alexa, 7 Display, 16 GB, Blue - with Special Offers',
       'All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi, 16 GB - Includes Special Offers, Black',
       'Fire Tablet, 7 Display, Wi-Fi, 16 GB - Includes Special Offers, Black',
       'Fire HD 8 Tablet with Alexa, 8 HD Display, 16 GB, Tangerine - with Special Offers',
       'Fire Tablet with Alexa, 7 Display, 16 GB, Magenta - with Special Offers',
       'All-New Fire HD 8 Tablet with Alexa, 8 HD Display, 16 GB, Marine Blue - with Special Offers'],
      dtype=object)